# Prepare Callbacks

Callbacks enhances training, monitoring progress, and ensuring best model is saved. 

Some common use cases for callbacks:

1. **Model Checkpointing:**
   - **Purpose:** Save the model's weights during training.
   - **Advantage:** Ensures that the best model (based on a chosen metric) is saved, allowing you to resume training or use the model for inference later.
   - **Example Callback:** `ModelCheckpoint`

2. **Early Stopping:**
   - **Purpose:** Halt training if a certain condition (e.g., no improvement in validation loss) is met.
   - **Advantage:** Prevents overfitting and saves training time.
   - **Example Callback:** `EarlyStopping`

3. **Learning Rate Scheduling:**
   - **Purpose:** Adjust the learning rate during training to optimize convergence.
   - **Advantage:** Helps fine-tune the model by dynamically adapting the learning rate.
   - **Example Callback:** Custom callback or `ReduceLROnPlateau`

4. **TensorBoard Integration:**
   - **Purpose:** Log training metrics for visualization in TensorBoard.
   - **Advantage:** Provides interactive visualizations of metrics like loss and accuracy.
   - **Example Callback:** `TensorBoard`

5. **Custom Logging and Monitoring:**
   - **Purpose:** Log custom metrics or information during training.
   - **Advantage:** Allows you to track specific aspects of the training process not covered by default metrics.
   - **Example Callback:** Custom callback for logging

6. **Data Augmentation Monitoring:**
   - **Purpose:** Monitor the effects of data augmentation on model performance.
   - **Advantage:** Provides insights into how augmentation impacts the training process.
   - **Example Callback:** Custom callback for visualizing augmented images

7. **Gradient Clipping:**
   - **Purpose:** Limit the magnitude of gradients during training.
   - **Advantage:** Helps prevent exploding gradients and stabilizes training.
   - **Example Callback:** Custom callback for gradient clipping

These callbacks collectively contribute to efficient training, model selection, and performance monitoring in a deep learning image classification pipeline. They enhance the adaptability of the pipeline to various scenarios and improve the overall training experience.

## Summary of key points about using callbacks


| **Callback**               | **Advantages**                                           | **Disadvantages**                                       | **When to Use**                                                                                                                                                                                                                                                                                             | **When Not to Use**                                                                                                                                                                                                |
|------------------------|---------------------------------------------------------|---------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Model Checkpoint      | - Saves the best model weights.                        | - Adds storage overhead for saving multiple checkpoints. | - Use when you want to save the best model during training for later use in inference or further training.                                                                                                                                     | - When storage constraints are severe, and saving multiple checkpoints is not feasible.                                                                                                                           |
| Early Stopping           | - Prevents overfitting by stopping training early.     | - May stop training too early, missing potential gains.  | - Use when you want to automatically stop training when a specified metric (e.g., validation loss) stops improving, preventing overfitting.                                                                                               | - When you want to run the training process for a fixed number of epochs, ignoring early stopping criteria.                                                                                                      |
| Learning Rate Schedule | - Dynamically adjusts learning rate during training.   | - Choosing an inappropriate schedule can harm convergence. | - Use when you want to adaptively adjust the learning rate to optimize convergence, especially in scenarios with varying gradients or non-uniform data.                                                                                    | - When learning rate tuning is unnecessary or the learning rate is fixed throughout training.                                                                                                                   |
| TensorBoard            | - Provides interactive visualizations of training metrics. | - Adds some computational overhead for logging.          | - Use when you want to monitor and visualize metrics (e.g., loss, accuracy) in real-time using TensorBoard.                                                                                                                                   | - When visualization of training metrics is not a priority, or computational resources are limited.                                                                                                            |
| Custom Logging         | - Allows logging of custom metrics or information.       | - Requires additional coding for custom logging logic.  | - Use when you need to track specific information or metrics not covered by default callbacks.                                                                                                                                                 | - In standard cases where default logging is sufficient, and there is no need for additional custom logging.                                                                                                  |
| Data Augmentation Monitoring | - Visualizes the effects of data augmentation.     | - May increase the complexity of monitoring.            | - Use when you want insights into how data augmentation impacts model training, helping to assess its effectiveness.                                                                                                                         | - When data augmentation has minimal impact on model performance or monitoring its effects is not a priority.                                                                                                |
| Gradient Clipping      | - Stabilizes training by limiting gradient magnitudes.  | - May affect convergence if clipping threshold is too low. | - Use when dealing with exploding gradient issues during training, preventing instability.                                                                                                                                                | - In cases where gradients are well-behaved and clipping is unnecessary.                                                                                                                                              |


In [1]:
%pwd

'/media/manralai/Drive D/ImageClassifier-Pipeline/research_env'

In [2]:
## to root
%cd ..

/media/manralai/Drive D/ImageClassifier-Pipeline


In [10]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

### Configuraion Manager Code

In [11]:
import os
from imageClassifier.constants import *
from imageClassifier.utils import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(root_dir=Path(config.root_dir),
                                                         tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                                         checkpoint_model_filepath=Path(config.checkpoint_model_filepath))
        return prepare_callback_config

### Componens Code

In [15]:
import os
import time
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request


class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}",)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=str(self.config.checkpoint_model_filepath),save_best_only=True)

    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks,self._create_ckpt_callbacks]

#### Above Warning

In summary, above output suggests that TensorFlow is running on the CPU due to the absence of CUDA drivers for the GPU. Additionally, it provides information about optimization and warns about the unavailability of TensorRT, which is relevant for GPU-based inference optimizations on NVIDIA GPUs. If you intend to use the GPU, ensure that CUDA and cuDNN are properly installed and configured.

#### Defining Pipeline

In [16]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-01-06 13:08:38,217: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-06 13:08:38,227: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-06 13:08:38,230: INFO: common: Created directory at: artifacts]
[2024-01-06 13:08:38,234: INFO: common: Created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-01-06 13:08:38,238: INFO: common: Created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
